<a href="https://colab.research.google.com/github/HaaLeo/vague-requirements-scripts/blob/master/colab-notebooks/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classify requirements as vague or not using [ktrain](https://github.com/amaiya/ktrain) and tensorflow


## Install dependencies
*ktrain* requires TensorFlow 2.1. See [amaiya/ktrain#151](https://github.com/amaiya/ktrain/issues/151).
Further we install a forked version of eli5lib to gain insights in the model's decision process and some self built helper functions to preprocess MTurk result files.

In [1]:
!pip3 install -q tensorflow_gpu==2.1.0 ktrain==0.17.3
!pip3 install -q -U git+https://github.com/HaaLeo/vague-requirements-scripts
!pip3 install -q git+https://github.com/amaiya/eli5@tfkeras_0_10_1

Check versions and enable logging

In [2]:
import tensorflow as tf
import ktrain
assert tf.__version__ == '2.1.0'
assert ktrain.__version__ == '0.17.3'

import logging
import sys

logging.basicConfig(
    format='%(asctime)s [%(threadName)-20.20s] [%(levelname)-5.5s]  %(message)s',
    stream=sys.stdout,
    level=logging.DEBUG)

## Set Parameters
Set the parameters for this run.
Ktrain ignores `max_features` and `ngram_range` in v0.17.3, see [amaiya/ktrain/issues#190](https://github.com/amaiya/ktrain/issues/190)

In [53]:
indices_to_read = [0,2,3,4] # indicate which MTurk files shall be read.
DATA_FILE_NAMES = [f'corpus-batch-{i}-mturk.csv' for i in indices_to_read]

RANDOM_STATE = 1 # for seeding

LEARNING_RATE = 5e-5
EPOCHS = 4
MODEL_NAME = 'distilbert-base-uncased'
MAX_LEN = 512
BATCH_SIZE = 6

# MAX_FEATURES = 35_000
# NGRAM_RANGE = 1

## Load Dataset

### Mount Google Drive
Mount the google drive to access the dataset

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


### Load Dataset Into Arrays

In [14]:
from vaguerequirementslib import read_csv_files, build_confusion_matrix, calc_majority_label
import pandas as pd

def read_drive_data(files_list: list, separator: str) -> pd.DataFrame:
    """
    Calculate the majority label for the given source file list

    Args:
        files_list (list): The CSV files to calculate the majority label for
        separator (str): The CSV separator
        drop_ties (bool): If there is a tie in votes (e.g.: One votes for vague one for not vague) then drop this entry from the confusion matrix.

    Returns:
        pd.DataFrame: The dataframe containing the majority label.
    """
    df = read_csv_files(files_list, separator)
    confusion_matrix = build_confusion_matrix(df, drop_ties=True)
    return calc_majority_label(confusion_matrix)

# Read all data
df = read_drive_data(
    [f'/content/drive/My Drive/datasets/corpus/labeled/{file_name}' for file_name in DATA_FILE_NAMES],
    ','
  )
df.head()


2020-07-04 13:27:11,333 [MainThread          ] [DEBUG]  Read file="/content/drive/My Drive/datasets/corpus/labeled/corpus-batch-0-mturk.csv" with 200 rows.
2020-07-04 13:27:11,348 [MainThread          ] [DEBUG]  Read file="/content/drive/My Drive/datasets/corpus/labeled/corpus-batch-2-mturk.csv" with 194 rows.
2020-07-04 13:27:11,363 [MainThread          ] [DEBUG]  Read file="/content/drive/My Drive/datasets/corpus/labeled/corpus-batch-3-mturk.csv" with 198 rows.
2020-07-04 13:27:11,378 [MainThread          ] [DEBUG]  Read file="/content/drive/My Drive/datasets/corpus/labeled/corpus-batch-4-mturk.csv" with 196 rows.
2020-07-04 13:27:11,395 [MainThread          ] [INFO ]  Build confusion matrix.
2020-07-04 13:27:11,563 [MainThread          ] [INFO ]  Dropped 180 requirements due to ties.
2020-07-04 13:27:11,565 [MainThread          ] [INFO ]  Built confusion matrix including 214 of 394 requirements. 
2020-07-04 13:27:11,570 [MainThread          ] [INFO ]  Overall "vague" votes count = 9

,requirement,vague_count,not_vague_count,majority_label
0,A fallback per band feature set resulting from...,2,0,1
1,Actuation of steering shall be possible regard...,0,2,0
2,"Additionally, the ZigBee end device shall then...",0,2,0
3,"Additionally, the plan provides traceability f...",2,0,1
4,"After completion of release of the resources, ...",0,2,0


### Split data set


In [15]:
from sklearn.model_selection import train_test_split
from typing import Tuple, List


def split_dataset(dataframe: pd.DataFrame) -> Tuple[List[str], List[int], List[str], List[int], List[str], List[int]]:
    """
    Split the dataset into training, validation and test set.

    Args:
        data_frame (pd.DataFrame): The data frame to generate the data sets from.

    Returns:
        Tuple[List[str], List[int], List[str], List[int], List[str], List[int]]: x_train, y_train, x_val, y_val, x_test, y_test
    """
    train_df, val_test_df = train_test_split(df, test_size=0.2, random_state=RANDOM_STATE, stratify=df['majority_label'])
    val_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=RANDOM_STATE, stratify=val_test_df['majority_label'])

    print(f'Training dataset: vague count="{train_df.sum()["majority_label"]}", not vague count="{train_df.shape[0] - train_df.sum()["majority_label"]}"')
    print(f'Validation dataset: vague count="{val_df.sum()["majority_label"]}", not vague count="{val_df.shape[0] - val_df.sum()["majority_label"]}"')
    print(f'Test dataset: vague count="{test_df.sum()["majority_label"]}", not vague count="{test_df.shape[0] - test_df.sum()["majority_label"]}"')

    # text.texts_from_df(train_df, text_column='requirement', label_columns=['majority_label'],)
    x_train = list(train_df['requirement'])
    y_train = list(train_df['majority_label'])
    x_val = list(val_df['requirement'])
    y_val = list(val_df['majority_label'])
    x_test = list(test_df['requirement'])
    y_test = list(test_df['majority_label'])
    
    return x_train, y_train, x_val, y_val, x_test, y_test

# Split the data set
x_train, y_train, x_val, y_val, x_test, y_test = split_dataset(df)


Training dataset: vague count="38", not vague count="133"
Validation dataset: vague count="5", not vague count="16"
Test dataset: vague count="5", not vague count="17"


## STEP 1:  Preprocess Data and Create a Transformer Model

We will use [DistilBERT](https://arxiv.org/abs/1910.01108).

In [54]:
import ktrain
from ktrain import text
t = text.Transformer(MODEL_NAME, maxlen=MAX_LEN, class_names=['not-vague', 'vague']) # 0=not-vague 1=vague
train_data = t.preprocess_train(x_train, y_train)
val_data = t.preprocess_test(x_val, y_val)
test_data = t.preprocess_test(x_test, y_test)

model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=train_data, val_data=val_data, batch_size=BATCH_SIZE)

preprocessing train...
language: en
train sequence lengths:
	mean : 21
	95percentile : 40
	99percentile : 63
2020-07-04 14:50:55,513 [MainThread          ] [DEBUG]  Starting new HTTPS connection (1): s3.amazonaws.com:443
2020-07-04 14:50:55,697 [MainThread          ] [DEBUG]  https://s3.amazonaws.com:443 "HEAD /models.huggingface.co/bert/bert-base-uncased-vocab.txt HTTP/1.1" 200 0


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 22
	95percentile : 38
	99percentile : 46


preprocessing test...
language: en
test sequence lengths:
	mean : 21
	95percentile : 36
	99percentile : 38


2020-07-04 14:50:56,016 [MainThread          ] [DEBUG]  Starting new HTTPS connection (1): s3.amazonaws.com:443
2020-07-04 14:50:56,159 [MainThread          ] [DEBUG]  https://s3.amazonaws.com:443 "HEAD /models.huggingface.co/bert/distilbert-base-uncased-config.json HTTP/1.1" 200 0
2020-07-04 14:50:56,166 [MainThread          ] [DEBUG]  Starting new HTTPS connection (1): cdn.huggingface.co:443
2020-07-04 14:50:56,388 [MainThread          ] [DEBUG]  https://cdn.huggingface.co:443 "HEAD /distilbert-base-uncased-tf_model.h5 HTTP/1.1" 200 0


## STEP 2:  Train the Model

In [55]:
learner.fit_onecycle(LEARNING_RATE, EPOCHS)



begin training using onecycle policy with max lr of 5e-05...
Train for 29 steps, validate for 1 steps
Epoch 1/4
29/29 [==============================] - 30s 1s/step - loss: 0.5909 - accuracy: 0.7076 - val_loss: 0.5602 - val_accuracy: 0.7619
Epoch 2/4
29/29 [==============================] - 20s 695ms/step - loss: 0.5304 - accuracy: 0.7778 - val_loss: 0.5716 - val_accuracy: 0.7619
Epoch 3/4
29/29 [==============================] - 20s 692ms/step - loss: 0.4444 - accuracy: 0.7836 - val_loss: 0.7308 - val_accuracy: 0.7619
Epoch 4/4
29/29 [==============================] - 20s 692ms/step - loss: 0.2536 - accuracy: 0.8830 - val_loss: 0.5832 - val_accuracy: 0.7143


## STEP 3: Evaluate and Inspect the Model

In [21]:
test_result = learner.validate(class_names=t.get_classes(), val_data=test_data)
print(test_result)

              precision    recall  f1-score   support

   not-vague       0.80      0.24      0.36        17
       vague       0.24      0.80      0.36         5

    accuracy                           0.36        22
   macro avg       0.52      0.52      0.36        22
weighted avg       0.67      0.36      0.36        22

[[ 4 13]
 [ 1  4]]


Let's examine the validation example about which we were the most wrong.

In [19]:
learner.view_top_losses(n=1, preproc=t)

----------
id:21 | loss:2.63 | true:vague | pred:not-vague)



In [22]:
top_loss_req = x_test[21] # Requirement that produces top loss
print(top_loss_req)
print(y_test[21])

Distance between supports shall not exceed resin is adequate support.
1


This post talks more about computing than `alt.atheism` (the true category), so our model placed it into the only computing category available to it: `comp.graphics`

## STEP 3.1: Gather Results

Gather results and write them to the drive.

In [24]:
from vaguerequirementslib import TP, TN, FP, FN, calc_all_metrics
result_data = {
    'metrics':{
        'vague': {
            TP: test_result[1][1],
            FP: test_result[0][1],
            TN: test_result[0][0],
            FN: test_result[1][0]
        },
        'not_vague': {
            TP: test_result[0][0],
            FP: test_result[1][0],
            TN: test_result[1][1],
            FN: test_result[0][1]
        }
    },
    'misc': {   
        'data_files': DATA_FILE_NAMES,
        'random_state': RANDOM_STATE
    },
    'hyperparameter': {
        'learning_rate': LEARNING_RATE,
        'epochs': EPOCHS
        'model_name': MODEL_NAME,
        'max_len': MAX_LEN,
        'batch_size': BATCH_SIZE
        # 'max_features': MAX_FEATURES,
        # 'ngram_range': NGRAM_RANGE
    }
}
result_data['metrics']['not_vague'].update(calc_all_metrics(**result_data['metrics']['not_vague']))
result_data['metrics']['vague'].update(calc_all_metrics(**result_data['metrics']['vague']))
print(result_data)

{'metrics': {'vague': {'true_positive': 4, 'false_positive': 13, 'true_negative': 4, 'false_negative': 1, 'accuracy': 0.36363636363636365, 'precision': 0.23529411764705882, 'recall': 0.8, 'specificity': 0.23529411764705882, 'false_negative_rate': 0.2, 'false_positive_rate': 0.7647058823529411, 'f1_score': 0.3636363636363636}, 'not_vague': {'true_positive': 4, 'false_positive': 1, 'true_negative': 4, 'false_negative': 13, 'accuracy': 0.36363636363636365, 'precision': 0.8, 'recall': 0.23529411764705882, 'specificity': 0.8, 'false_negative_rate': 0.7647058823529411, 'false_positive_rate': 0.2, 'f1_score': 0.3636363636363636}}}


## STEP 4: Making Predictions on New Data in Deployment

In [25]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [26]:
predictor.predict(top_loss_req)

'not-vague'

In [27]:
# predicted probability scores for each category
predictor.predict_proba(top_loss_req)

array([0.9278868 , 0.07211317], dtype=float32)

In [28]:
predictor.get_classes()

['not-vague', 'vague']

As expected, `soc.religion.christian` is assigned the highest probability.

Let's invoke the `explain` method to see which words contribute most to the classification.



In [29]:
predictor.explain(top_loss_req)

The words in the darkest shade of green contribute most to the classification and agree with what you would expect for this example.

We can save and reload our predictor for later deployment.

In [ ]:
predictor.save('/tmp/my_distilbert_predictor')

In [ ]:
reloaded_predictor = ktrain.load_predictor('/tmp/my_distilbert_predictor')

In [ ]:
reloaded_predictor.predict('My computer monitor is really blurry.')

'comp.graphics'